<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# Описание


Мы  будем разрабатывать метод преобразования данных для защиты персональных данных клиентов страховой компании «Хоть потоп»

## Загрузка данных

1.1. Загрузим библиотеки и файл с данными

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression

In [2]:
try:
    clients = pd.read_csv('/Users/galina//Desktop/учёба/спринт 10/insurance.csv') 
except:
    clients = pd.read_csv('/datasets/insurance.csv')
display(clients.head(20))    

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


1.2. Предобработка данных

In [3]:
display(clients.info())  #полсмотрим информацию в имеющемся файле с данными (в том числе типы данных)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

In [4]:
#заменяю тип данных в столбцах Возраст и Зарплата на int, так как по сути это целые числа:
clients['Возраст'] = clients['Возраст'].astype('int')
clients['Зарплата'] = clients['Зарплата'].astype('int')
clients.dtypes

Пол                  int64
Возраст              int64
Зарплата             int64
Члены семьи          int64
Страховые выплаты    int64
dtype: object

In [5]:
clients.duplicated().sum() #посмотрим дубликаты

153

In [6]:
clients.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [7]:
clients.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.359400,1.194200,0.148000
std,0.500049,8.440807,9900.082063,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


# Вывод:
1. Изменила типы данных на более подходящие;
2. Количество дубликатов - 153, удалять не стала, так как тут, возможны одинаковые значения по разным клиентам;
3. Пропусков нет;
4. В исходном файле данные по мужчинам и женщинам представлены практически в равных долях, средний возраст рассматриваемых клиентов - 31 год, средняя зарплата - 39916.

## Умножение матриц

Проверим, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' =  P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$



Произведение $(P^T)^{-1} P^T$
 дает единичную матрицу, которую можно исключить из уравнения. Тогда формула принимает вид:
 
$w' = P^{-1}(X^T X)^{-1} X^T y$ 


$𝑤′=P^{-1}𝑤$

ч.т.д.

In [8]:
features = clients.drop('Страховые выплаты',axis=1)
target = clients['Страховые выплаты']

features.shape, target.shape

((5000, 4), (5000,))

In [9]:
#записываю формулу линейной регрессии с единичным столбцом:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1) 
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y          #считаем w по формуле минимизации MSE
print('            параметр w:', w[1:])

model = LinearRegression()                   
model.fit(features, target)                   
print('оценочные коэффициенты:', model.coef_) #вывожу оценочные коэффициенты для линейной регрессии

            параметр w: [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]
оценочные коэффициенты: [ 7.92580563e-03  3.57083050e-02 -1.70081903e-07 -1.35676627e-02]


параметр w и оценочные коэффициенты полностью совпадают, следовательно качество линейной регрессии не изменяется.

In [10]:
print('r2 для исходных признаков:', r2_score(y, model.predict(features))) 

random = np.random.normal(size=(4,4)) #генерирую случайную матрицу
np.linalg.inv(random)                 #проверяю матрицу на обратимость
features_new = features @ random      #изменяю матрицу признаков 
l = LinearRegression()               
l.fit(features_new, target)          
print('r2 для измененных признаков:', r2_score(y, l.predict(features_new)))

r2 для исходных признаков: 0.42494550308169177
r2 для измененных признаков: 0.424945503081698


r2 совпадают, также значения r2 находятся в промежутке от 0 до 1, соответственно линейная регрессия работает

# Вывод:

качество линейной регрессии при умножении признаков на обратимую матрицу не изменится, так как параметр w и оценочные коэффициенты, оценки r2 для исходных признаков и измененных совпадают

## Алгоритм преобразования

Так как, как мы увидели выше, качество линейной регрессии при умножении признаков на обратимую матрицу не изменится, для преобразования данных для защиты информации можно использовать следующий алгоритм:

1.Сгенерировать случайную квадратную матрицу random и умножить матрицу признаков на случайную квадратную матрицу random;

2.Для усиления защиты можно дополнительно умножить матрицу признаков на какую-то цифру. Например, 7.

## Проверка алгоритма

Используем предложенный алгоритм преобразования, посчитаем и сравним r2 для исходных и защищенных данных

In [11]:
random = np.random.normal(size=(4,4))   #генерирую случайную матрицу
np.linalg.inv(random)                   #проверяю матрицу на обратимость
features_new = features @ random * 7    #умножаю матрицу признаков на случайную  матрицу и на 7
features_new.head()

,0,1,2,3
0,340500.575954,-399108.245412,-41593.482707,-95871.482856
1,260804.751665,-305735.479724,-31805.940934,-73649.361489
2,144113.920660,-168947.936884,-17560.202803,-40745.515082
3,286330.136363,-335581.691983,-35029.757076,-80445.990473
4,179144.468301,-209994.882649,-21857.790088,-50521.595200


In [12]:
l = LinearRegression()               
l.fit(features_new, target)          
print('r2:',
      r2_score(target,
               l.predict(features_new))) #вычисляю r2 для измененных признаков

r2: 0.4249455030816969


r2 на защищенных данных совпадает с r2 на исходных данных, следовательно, алгоритм справляется с защитой данных

# Вывод:

Для защиты данных к качестве алгоритма преобразования можно использовать умножение матрицы признаков на случайную квадратную матрицу и на какое-то случайное число

# Общий вывод:

От нас требовалось защитить данные клиентов страховой компании «Хоть потоп» и разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом качество моделей машинного обучения не должно было ухудшиться.

В результате мы разработали алгоритм, где данные были преобразованы методом умножения признаков исходных данных на обратимую матрицу и на цифру.